In [1]:
import stormpy
import stormpy.info

In [2]:
print(stormpy.__version__)
print(stormpy.info.storm_version())

1.8.0
1.8.1


In [3]:
prism_program = stormpy.parse_prism_program("benchmarks/box-stochastic/box-stochastic.prism")
print(prism_program.model_type)

PrismModelType.MDP


In [4]:
options = stormpy.BuilderOptions(True, True)
options.set_build_state_valuations()
options.set_build_choice_labels()
model = stormpy.build_sparse_model_with_options(prism_program, options)
print("Number of states: {}".format(model.nr_states))
print("Number of transitions: {}".format(model.nr_transitions))
print("Labels: {}".format(model.labeling.get_labels()))

Number of states: 10098
Number of transitions: 88506
Labels: {'target', 'deadlock', 'init'}


In [5]:
formula_str = 'Pmax=? [F "target"]'
properties = stormpy.parse_properties(formula_str, prism_program)

result = stormpy.model_checking(model, properties[0])

initial_state = model.initial_states[0]
print("Min probability to reach target is {}".format(result.at(initial_state)))

Min probability to reach target is 0.9655172413793076


In [6]:
state = model.states[0]
state_vals = model.state_valuations
print("State {} with variable values: {}".format(state, state_vals.get_string(state.id)))

choice_labels = model.choice_labeling
for action in state.actions:
    for transition in action.transitions:
        print("With action {} and probability {}, go to state {} {}".format(choice_labels.get_labels_of_choice(action.id), transition.value(), transition.column, state_vals.get_string(transition.column)))
        if transition.column > 10: break

State 0 with variable values: [x=1	& y=50]
With action {'R'} and probability 0.7999999999999999, go to state 1 [x=2	& y=50]
With action {'R'} and probability 0.1, go to state 2 [x=1	& y=51]
With action {'R'} and probability 0.1, go to state 3 [x=1	& y=49]
With action {'U'} and probability 0.1, go to state 1 [x=2	& y=50]
With action {'U'} and probability 0.7999999999999999, go to state 2 [x=1	& y=51]
With action {'U'} and probability 0.1, go to state 3 [x=1	& y=49]
With action {'D'} and probability 0.1, go to state 1 [x=2	& y=50]
With action {'D'} and probability 0.1, go to state 2 [x=1	& y=51]
With action {'D'} and probability 0.7999999999999999, go to state 3 [x=1	& y=49]


In [7]:
def string_val_to_tuple(state_val):
    values = state_val[1:-1].split("\t&")
    x = int(values[0].split("=")[1])
    y = int(values[1].split("=")[1])
    return x, y

In [8]:
result = stormpy.model_checking(model, properties[0], extract_scheduler=True)
scheduler = result.scheduler
assert scheduler.memoryless
assert scheduler.deterministic

In [9]:
buttons = [['' for _ in range(101)] for _ in range(101)]
for state in model.states:
    x, y = string_val_to_tuple(state_vals.get_string(state.id))
    if not ((0 <= x <= 100) and (0 <= y <= 100)):
        break
    choice = scheduler.get_choice(state)
    action = list(choice_labels.get_labels_of_choice(choice.get_deterministic_choice()))[0]
    buttons[x][y] = action

In [12]:
for y in range(99, 0, -1):
    line = ""
    for x in range(1, 100):
        line += buttons[x][y] + ""

    print(line)

DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD


In [11]:
buttons[50][28]

'U'